In [1]:
# ========================================
# SETUP CELL - Structure Merging Tool
# ========================================

# Import required libraries
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import subprocess
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up paths
ROOT_DIR = Path('.')
MERGING_DIR = ROOT_DIR / 'structure_merging_model'
INPUTS_DIR = ROOT_DIR / 'accel_tlnd_model' / 'inputs'
MERGE_SCRIPT = MERGING_DIR / 'merge_structures.py'

# ========================================
# UTILITY FUNCTIONS
# ========================================

def get_available_input_files():
    """Get list of available parquet input files for merging"""
    available_files = []
    
    # Only check structure_merging_model/inputs directory
    search_dir = MERGING_DIR / 'inputs'
    
    if search_dir.exists():
        # Only look for parquet files
        for file_path in search_dir.glob('**/*.parquet'):
            relative_path = file_path.relative_to(ROOT_DIR)
            available_files.append(str(relative_path))
    
    # Sort files for better organization
    available_files.sort()
    
    # Add a custom option at the beginning
    available_files.insert(0, "-- Select a file or type custom path --")
    
    return available_files

def create_merging_widgets():
    """Create all UI widgets for structure merging"""
    
    # Get available input files
    available_files = get_available_input_files()
    
    # File selection dropdown
    file_dropdown = widgets.Dropdown(
        options=available_files,
        value=available_files[0],
        description='Quick Select:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='450px')
    )
    
    # Input file path textarea (copy/paste friendly)
    input_file_text = widgets.Textarea(
        value='structure_merging_model/inputs/example_parish_structures.parquet',
        description='Input File:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px', height='60px'),
        placeholder='Type or paste parquet file path here',
        rows=2
    )
    
    # Output file path textarea (copy/paste friendly)  
    output_file_text = widgets.Textarea(
        value='merged_structures.parquet',
        description='Output File:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='500px', height='60px'),
        placeholder='Type or paste output parquet file path here',
        rows=2
    )
    
    # Merging radius parameter
    radius_widget = widgets.FloatText(
        value=20.0,
        description='Merging Radius (m):',
        style={'description_width': 'initial'},
        min=1.0,
        max=1000.0
    )
    
    # Connect dropdown and text
    def on_dropdown_change(change):
        if change['new'] != "-- Select a file or type custom path --":
            input_file_text.value = change['new']
    
    file_dropdown.observe(on_dropdown_change, names='value')
    
    # Input file widget wrapper
    class InputFileWrapper:
        def __init__(self, text_widget):
            self.text = text_widget
        @property
        def value(self):
            return self.text.value.strip()
    
    input_file_widget = InputFileWrapper(input_file_text)
    
    return {
        'input_file': input_file_widget,
        'input_dropdown': file_dropdown,
        'input_text': input_file_text,
        'output_file': output_file_text,
        'radius': radius_widget
    }

print("✅ Structure Merging Tool setup completed!")
print(f"📁 Working directory: {ROOT_DIR.absolute()}")
print(f"🔧 Merge script: {MERGE_SCRIPT}")
print("🎯 Now run the Merging UI cell below!")


✅ Structure Merging Tool setup completed!
📁 Working directory: /Users/wuyuezi/Desktop/energy_system_design_tools/accel_tlnd_training
🔧 Merge script: structure_merging_model/merge_structures.py
🎯 Now run the Merging UI cell below!


In [2]:
# ========================================
# MERGING UI CELL
# ========================================

# Create all UI widgets
widgets_dict = create_merging_widgets()

# Create buttons
refresh_button = widgets.Button(
    description="🔄 Refresh",
    button_style='info',
    layout=widgets.Layout(width='120px')
)

run_button = widgets.Button(
    description="🔗 Run Merging",
    button_style='primary',
    icon='play'
)

# Create output area
merging_output = widgets.Output()

# Button click handlers
def on_refresh_click(b):
    try:
        files = get_available_input_files()
        widgets_dict['input_dropdown'].options = files
        print(f"✅ Found {len(files)-1} input files")
    except Exception as e:
        print(f"⚠️ Error: {e}")

def on_run_click(b):
    with merging_output:
        clear_output()
        
        # Get parameters
        input_file = widgets_dict['input_file'].value
        output_file = widgets_dict['output_file'].value.strip()
        radius = widgets_dict['radius'].value
        
        print("🔗 Starting structure merging process...")
        print(f"📁 Input file: {input_file}")
        print(f"📤 Output file: {output_file}")
        print(f"📏 Merging radius: {radius} meters")
        print("")
        
        try:
            # Check if input file exists
            input_path = Path(input_file)
            if not input_path.exists():
                print(f"❌ Input file not found: {input_file}")
                return
            
            # Run the merging script
            result = subprocess.run(
                [sys.executable, str(MERGE_SCRIPT), 
                 '--input_file', input_file,
                 '--output_file', output_file,
                 '--radius', str(radius)],
                capture_output=True,
                text=True,
                cwd=ROOT_DIR
            )
            
            if result.returncode == 0:
                print("✅ Merging completed successfully!")
                print("")
                
                # Parse and display the output in a nice format
                output_lines = result.stdout.strip().split('\n')
                before_count = None
                after_count = None
                processing_time = None
                
                for line in output_lines:
                    if 'Before merging:' in line:
                        before_count = line.split(': ')[1].split(' ')[0]
                    elif 'After merging:' in line:
                        after_count = line.split(': ')[1].split(' ')[0]
                    elif 'Processing time:' in line:
                        processing_time = line.split(': ')[1]
                
                # Display formatted summary
                print("📊 MERGING RESULTS SUMMARY")
                print("=" * 50)
                if before_count:
                    print(f"🏠 Before merging: {before_count} structures")
                if after_count:
                    print(f"🔗 After merging: {after_count} structures")
                if processing_time:
                    print(f"⏱️ Processing time: {processing_time}")
                print("=" * 50)
                
                print("\n✅ Results saved successfully!")
                print(f"📁 Output file: {output_file}")
                
            else:
                print("❌ Merging failed!")
                print("\n🔍 Error details:")
                print(result.stderr)
                
        except Exception as e:
            print(f"❌ Error running merging process: {e}")

# Connect buttons to handlers
refresh_button.on_click(on_refresh_click)
run_button.on_click(on_run_click)

# ========================================
# DISPLAY THE UI
# ========================================

display(HTML("<h2>🔗 Structure Merging Tool</h2>"))

display(HTML("<h3>📁 Input/Output Configuration</h3>"))
display(HTML("""
<div style='color:#555; font-size:0.9em; margin-bottom:6px;'>
<b>💡 Tip:</b> Use the dropdown to quickly select available parquet files, or copy/paste custom paths in the text fields below.<br>
<b>Note:</b> Input files must be in parquet format and located in structure_merging_model/inputs directory.
</div>
"""))
display(widgets.HBox([widgets_dict['input_dropdown'], refresh_button]))
display(widgets_dict['input_text'])
display(widgets_dict['output_file'])

display(HTML("<h3>⚙️ Merging Parameters</h3>"))
display(HTML("""
<div style='color:#555; font-size:0.9em; margin-bottom:6px;'>
<b>Merging Radius:</b> Structures within this distance (in meters) will be merged into compound-level points.<br>
</div>
"""))
display(widgets_dict['radius'])

display(HTML("<h3>🚀 Execute Merging</h3>"))
display(run_button)
display(merging_output)

print("🎉 Structure Merging UI is ready!")
print("📋 Configure your input file, output path, and merging radius, then click 'Run Merging'")
print("📊 The tool will show before/after statistics when completed.")


Textarea(value='structure_merging_model/inputs/example_parish_structures.parquet', description='Input File:', …

Textarea(value='merged_structures.parquet', description='Output File:', layout=Layout(height='60px', width='50…

FloatText(value=20.0, description='Merging Radius (m):', style=DescriptionStyle(description_width='initial'))

Button(button_style='primary', description='🔗 Run Merging', icon='play', style=ButtonStyle())

Output()

🎉 Structure Merging UI is ready!
📋 Configure your input file, output path, and merging radius, then click 'Run Merging'
📊 The tool will show before/after statistics when completed.
